In [102]:
import pandas as pd
import folium as fl
from folium.plugins import MarkerCluster
import os
import geopandas

In [14]:
def get_color(height):
    if (height<1000):
        return "green"
    elif (height<3000):
        return "orange"
    else:
        return "red"

In [103]:
map1 = fl.Map(location=[37.296933,-121.9574983], zoom_start=8, tiles='Stamen Terrain')

volcanoes = pd.read_csv("folium/Volcanoes_USA.txt")

marker_cluster = MarkerCluster().add_to(map1)
volcanoes.apply(lambda x: fl.CircleMarker(location=[x.LAT,x.LON], radius=9, 
                                          popup=' '.join([x.NAME,str(int(x.ELEV)),'m']),color=get_color(x.ELEV)
                                         ).add_to(marker_cluster),
                axis=1)

map1.save("folium/volcanoes.html")
map1

In [176]:
unemployment_map = fl.Map(location=[48, -102], zoom_start=3)

counties_geo_path = os.path.join("folium","us-counties.json")
counties_unemployment = os.path.join("folium","us-counties-unemployment.json")
states_codes_wiki_url = "https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations"

state_codes = pd.read_html(states_codes_wiki_url)[0].iloc[12:,4:6]
state_codes = state_codes.dropna(axis=0).rename({4: "Number", 5: "Code"},axis=1)

def get_name(s):
    s = s.rsplit(None,2)[0]
    if s.endswith("Census"):
        s = s.rsplit(None,1)[0]
    return s

counties_data = pd.read_json(counties_unemployment).iloc[:3171,:]
counties_data['state'] = counties_data.name.map(lambda x: x[-2:])
counties_data['code'] = counties_data.code.map(lambda x: x[-3:])
counties_data = pd.merge(counties_data,state_codes, left_on="state", right_on="Code", how="left")
counties_data['name'] = counties_data.name.map(get_name)
counties_data = counties_data.drop(['code','state','Number','Code'], axis='columns')

fl.Choropleth(geo_data=counties_geo,
              data=counties_data,
              columns=('name','value'),
              key_on='feature.properties.name',
              fill_color='YlOrRd',
              fill_opacity=0.7,
              line_opacity=0.2
             ).add_to(unemployment_map)

unemployment_map.save(os.path.join('folium','unemployment_by_county.html'))
unemployment_map